In [ ]:
import numpy as np
import pandas as pd

# 学生とゼミのマッチング度を計算し、スコアの高い順にゼミを推薦する
def radar(sheet_name, index):
    zemidf = pd.read_excel('./公開用テストデータ.xlsx', sheet_name='ゼミ情報', index_col=0)
    studentdf = pd.read_excel('./公開用テストデータ.xlsx', sheet_name='学生架空データ')
    ratiodf = pd.read_excel('./公開用テストデータ.xlsx', sheet_name=sheet_name)

    # 学生データの取得
    student_name = studentdf.loc[index, '学生']
    answer = studentdf.iloc[index]

    # 計算用の初期化配列
    total_scores = []
    zemi_values =[]

    #  各ゼミに対してスコア計算
    for i in zemidf.index:

        total_score = 0
        answer_count = 0
        zemi_value_sum = 0

        # 興味のある項目をカウント（選択肢が3以上）
        for _, row in ratiodf.iterrows():
            question = row["question"]
            if answer[question] >= 3:
                answer_count += 1

        #ゼロ除算の防止とルート化
        if answer_count == 0:
            answer_count = 1
        answer_count = np.sqrt(answer_count)

        # 各質問項目に対して差分計算
        for _, row in ratiodf.iterrows():
            question = row["question"]
            ratio = row["ratio"]

            student_answer = answer[question]
            zemi_value = zemidf.loc[i, question]
            zemi_value_sum += zemi_value
            
            # ゼミの回答が2未満の場合は計算対象外
            if zemi_value < 2:
                continue

            # 差分と調整
            difference = ((student_answer - zemi_value) ** 2) / answer_count

            # 近似度が高い場合の補正
            if difference <= 1:
                difference *= 0.9

            # 完全一致のボーナス
            if student_answer == zemi_value:
                total_score += 5

            # 重み付けして合計スコアに加算
            total_score += difference * ratio

        # スコアを記録
        zemidf.loc[i, "score"] = total_score
        total_scores.append(total_score)

        
    # スコアの正規化（平均50、標準偏差10）
    scores = np.array(total_scores)
    zemis = np.array(zemi_values)

    if len(scores) > 1:
        mean_score = np.mean(scores)
        std_dev = np.std(scores)

        if std_dev != 0:
            z_scores = (scores - mean_score) / std_dev
            normalized_scores = 50 + (z_scores * 10)
            normalized_scores = np.clip(normalized_scores, 0, 100)
            
            # スコアが低い時の調整
            if all(normalized_scores) < 70:
                normalized_scores = normalized_scores + 20
        else:
            normalized_scores = np.full_like(scores, 50)
    else:
        normalized_scores = scores


    # 学生の回答とゼミの回答が全て一致していた場合のみ、正規化した点数を100点とする 
    for idx, (i, row) in enumerate(zemidf.iterrows()):
        all_match = all(
            question in answer and str(answer[question]) != "" and str(answer[question]) == str(row[question])
            for question in ratiodf["question"]
            if question in row
        )
        
        if all_match:
            normalized_scores[idx] = 100

    # 正規化スコアを保存
    zemidf["normalized_score"] = normalized_scores


    # 上位10件を表示
    top10 = zemidf.sort_values('normalized_score', ascending=False)
    columns_to_display = ["score", "normalized_score"]
    display(top10.loc[top10.index[:10], columns_to_display])
    print(student_name)


# ソートされたデータフレームを表示
for i in range(15):
    radar('重み付け', i)

,score,normalized_score
ゼミ情報,,
Iゼミ,-31.715476,92.168229
Cゼミ,-64.765321,78.573004
Bゼミ,-67.287302,77.535574
Jゼミ,-86.964700,69.441175
Dゼミ,-91.150923,67.719150
Hゼミ,-91.150923,67.719150
Eゼミ,-92.126001,67.318047
Fゼミ,-96.150923,65.662374
Aゼミ,-117.375224,56.931648


学生1


,score,normalized_score
ゼミ情報,,
Eゼミ,-28.174457,90.101831
Aゼミ,-58.262379,76.702357
Dゼミ,-64.076156,74.113227
Cゼミ,-64.237457,74.041392
Bゼミ,-72.126001,70.528277
Hゼミ,-73.628942,69.858951
Fゼミ,-78.101078,67.867313
Gゼミ,-81.150923,66.509083
Iゼミ,-94.486680,60.570084


学生2


,score,normalized_score
ゼミ情報,,
Cゼミ,-98.0,83.060833
Eゼミ,-103.0,81.981425
Dゼミ,-124.0,77.447913
Fゼミ,-129.0,76.368505
Hゼミ,-133.5,75.397038
Bゼミ,-158.0,70.107941
Jゼミ,-193.0,62.552087
Gゼミ,-202.0,60.609153
Iゼミ,-202.0,60.609153


...

学生13


,score,normalized_score
ゼミ情報,,
Cゼミ,-63.5,84.615359
Bゼミ,-86.5,80.203899
Iゼミ,-92.5,79.053083
Dゼミ,-93.0,78.957182
Hゼミ,-133.5,71.189176
Eゼミ,-143.5,69.271150
Jゼミ,-174.0,63.421171
Gゼミ,-183.0,61.694947
Aゼミ,-208.0,56.899882


学生14


,score,normalized_score
ゼミ情報,,
Eゼミ,-22.0,89.874972
Dゼミ,-74.0,78.353249
Bゼミ,-77.0,77.688534
Hゼミ,-93.0,74.143389
Cゼミ,-98.0,73.035531
Fゼミ,-123.0,67.496241
Aゼミ,-136.5,64.505024
Jゼミ,-158.5,59.630449
Gゼミ,-167.5,57.636305


学生15
